In [ ]:
from __future__ import unicode_literals, print_function, division 
from io import open 
import pandas as pd
import string
import re
import unicodedata 
import random 
import matplotlib.pyplot as plt
#plt.switch_backend("agg")
import matplotlib.ticker as ticker 
import numpy as np
import time
import math

#!pip install thai_tokenizer 
#from thai_tokenizer import Tokenizer
#from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(123456789)

epochs=2
epoch=1
nr_pairs_subset=10**(4) 
warmup_steps=10**(2)
#nr_pairs_subset=len(pairs) 
emb_size=512
nhead=8
#ffn_hid_dim=256
#batch_size=32
num_encoder_layers=4
num_decoder_layers=4

file1_text = "/content/drive/MyDrive/WunderSchildt/Translation/Prints/Char rus CV transformer, epoch %d, names %d,embedding %d.txt" % (epochs,nr_pairs_subset,emb_size)
file1 = open(file1_text, "w")


##############3
#Functions that transform the words in numbers. Needs a tokenizer since in certain languages 
#1 english word=n other language words 
###############

src_lang="En"
tgt_lang="Rus"


unk_idx=0
pad_idx=1
bos_idx=2
eos_idx=3

class PositionalEncoding(nn.Module):
    def __init__(self,emb_size:int,dropout:float,maxlen=5000):
        super().__init__()
        den=torch.exp(-torch.arange(0,emb_size,2)*math.log(10000)/emb_size)
        pos=torch.arange(0,maxlen).reshape(maxlen,1)
        pos_embedding=torch.zeros((maxlen,emb_size))
        pos_embedding[:,0::2]=torch.sin(pos*den)
        pos_embedding[:,1::2]=torch.cos(pos*den)
        pos_embedding=pos_embedding.unsqueeze(-2)
        
        self.dropout=nn.Dropout(dropout)
        self.register_buffer("pos_embedding",pos_embedding)
        
    def forward(self,token_embedding):
        return self.dropout(token_embedding+self.pos_embedding[:token_embedding.size(0),:])
        
class TokenEmbedding(nn.Module):
    def __init__(self,vocab_size,embed_size):
        super().__init__()
        self.embedding=nn.Embedding(vocab_size,emb_size)
        self.emb_size=emb_size
        
    def forward(self,tokens):
        return self.embedding(tokens.long())*math.sqrt(self.emb_size)
        
        
class seq2seqTransformer(nn.Module):
    def __init__(self,num_encoder_layers,num_decoder_layers,emb_size,nhead,src_vocab_size,tgt_vocab_size,
                dim_feedforward=512,dropout=0.1):
        super().__init__()
        self.transformer=nn.Transformer(d_model=emb_size,nhead=nhead,num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,dim_feedforward=dim_feedforward,dropout=dropout)
        
        self.generator=nn.Linear(emb_size,tgt_vocab_size)
        self.src_tok_emb=TokenEmbedding(src_vocab_size,emb_size)
        self.tgt_tok_emb=TokenEmbedding(tgt_vocab_size,emb_size)
        self.positional_encoding=PositionalEncoding(emb_size,dropout)
        
    def forward(self,src,tgt,src_mask,tgt_mask,src_padding_mask,tgt_padding_mask,memory_key_padding_mask):
        src_emb=self.positional_encoding(self.src_tok_emb(src))
        tgt_emb=self.positional_encoding(self.tgt_tok_emb(tgt))
        out=self.transformer(src_emb,tgt_emb,src_mask,tgt_mask,None,src_padding_mask,tgt_padding_mask,memory_key_padding_mask) 
        out=self.generator(out)
        return out
    
    def encode(self, src, src_mask):
        return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)),src_mask)
    
    def decode(self,tgt,memory,tgt_mask):
        return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)),memory,tgt_mask)
        

def generate_square_mask(size):
    mask=(torch.triu(torch.ones((size,size),device=device))==1).transpose(0,1)   #make lower triangular with 1's
    #make suare matrix in upper triangle has -inf and in the lower triangle it has 0.0
    mask=mask.float().masked_fill(mask==0,float("-inf")).masked_fill(mask==1,float(0.0))   
    return mask

class Lang:
    def __init__(self,lang):
        self.lang=lang
        self.char2index={}
        self.char2count={}
        self.index2char={0: "<unk>", 1:"<pad>", 2:"<bos>", 3:"<eos>" }
        self.n_chars=4
        
    def addName(self,sentence):
        for char in sentence:
            self.addChar(char.lower())
            
    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char]=self.n_chars
            self.char2count[char]=1
            self.index2char[self.n_chars]=char
            self.n_chars+=1
        else:
            self.char2count[char]+=1 
            
token_transform = {}
def timming(since):
    now=time.time()
    s=now-since
    m=int(s/60)
    s=s-m*60
    return "%dm %ds"%(m,s)
def unicodeToAscii(s):
        return ''.join(c for c in unicodedata.normalize("NFD",s) if unicodedata.category(c)!="Mn")
def normalizeString(string):
    string=unicodeToAscii(string.lower().strip())
    string=re.sub(r"([.!?])",r" \1",string)
    string=string.replace(",","")
    return string

def readLangs(src_lang,tgt_lang,reverse):
    
    forbidden_chars=['\u200e','\u200b','\u200f','ª',')','|','[',']','ǃ','$','‐','?','+','̱','̄','̜','̕','︡', '︠','̈',"ʿ",'"',"ʹ","ʼ","‘","′","'́'","«","»","\xad","\\","."]
    apostrophes=["`","’","ʻ","ʾ","'"]
    chirilic=["А", "Б", "В", "Г", "Д", "Е", "Ё", "Ж", "З", "И", "Й", "К", "Л", "М", "Н", "О", "П", "Р", "С", "Т", "У", "Ф", "Х", "Ц", "Ч", "Ш", "Щ", "Ъ", 
              "Ы", "Ь", "Э", "Ю", "Я"]
    chirilic=[char.lower() for char in chirilic]
    sh=["ş","ṣ","ș"]

    print("reading lines...")
    file1.write("reading lines... \n")
    
    lines=open("/content/drive/MyDrive/WunderSchildt/Translation/Data/%s-%s Names.txt"%(src_lang,tgt_lang),encoding="UTF-8").read().strip().split("\n")
    #lines=open("./Data/%s-%s_Names.txt"%(src_lang,tgt_lang),encoding="UTF-8").read().strip().split("\n")
    pairs=[]
    correct_line_idx=[] 
    i=0
    while i<=len(lines)-1:
      flag=1
      #Keep turkish, etc names or not?
      #lines[i]=unicodeToAscii(lines[i])
      if "-" in lines[i]:
        lines[i]=lines[i].replace("-","-")
      if "–" in lines[i]:
        lines[i]=lines[i].replace("–","-")
      for forbidden_char in forbidden_chars:
        if forbidden_char in lines[i]:
          lines[i]=lines[i].replace(forbidden_char,"")
      for apos in apostrophes:
        if apos in lines[i]:
          lines[i]=lines[i].replace(apos,"'") 
      for s in sh:
        if s in lines[i]:
          lines[i]=lines[i].replace(s,"ş")
      
      if reverse==False:
        s_input=lines[i].split("\t")[0]
        s_output=lines[i].split("\t")[1]
        temp_output=s_output.replace(" ","").replace(",","").replace("-","")
        temp_output=[char.lower() for char in temp_output]
        for char in temp_output:
          if char not in chirilic:
            flag=0
            break
      else:
        s_output=lines[i].split("\t")[0]
        s_input=lines[i].split("\t")[1]
        temp_input=s_input.replace(" ","").replace(",","").replace("-","")
        temp_input=[char.lower() for char in temp_input]
        for char in temp_input:
          if char not in chirilic:
            flag=0
            break

      if flag==0:
        i+=1
      else:
        correct_line_idx.append(i)
        if "," in lines[i]:
          if "," in s_output:
            temp=s_output.split(",")
            temp=[word.replace(" ","") for word in temp]
            temp=[temp[len(temp)-i] for i in range(1,len(temp)+1)]
            s_output=" ".join(temp)
            s_output=[char.lower() for char in s_output]
          if "," in s_input:
            temp=s_input.split(",")
            temp=[word.replace(" ","") for word in temp]
            temp=[temp[len(temp)-i] for i in range(1,len(temp)+1)]
            s_input=" ".join(temp) 
            s_input=[char.lower() for char in s_input]    
          pairs.append([s_input,s_output])
        else: 
          s_input=[char.lower() for char in s_input]
          s_output=[char.lower() for char in s_output]
          pairs.append([s_input,s_output])
        i+=1

    if reverse==False:
        input_lang=Lang(src_lang)
        output_lang=Lang(tgt_lang)
    else:
        #pairs=[list(reversed(p)) for p in pairs]
        input_lang=Lang(tgt_lang)
        output_lang=Lang(src_lang) 
        
    return input_lang, output_lang, pairs, correct_line_idx


def prepareData(src_lang,tgt_lang,reverse):
    input_lang, output_lang, pairs, correct_line_idx=readLangs(src_lang,tgt_lang,reverse)
    #token_transform[src_lang] = get_tokenizer('spacy', language='en_core_web_sm')
    #token_transform[tgt_lang] = get_tokenizer('spacy', language='rus')
    print("Read %s name pairs"%len(pairs))
    file1.write("Read %s name pairs \n"%len(pairs))
    print("Counting characters:")
    file1.write("Counting characters: \n")
    for pair in pairs:      
        input_lang.addName(pair[0])
        output_lang.addName(pair[1]) 
        
    print("counted characters:")
    file1.write("counted characters: \n")
    print(input_lang.lang,input_lang.n_chars)
    file1.write(str(input_lang.lang)+":"+str(input_lang.n_chars)+"\n") 
    print(output_lang.lang,output_lang.n_chars) 
    file1.write(str(output_lang.lang)+":"+str(output_lang.n_chars)+"\n") 
    
    return input_lang, output_lang, pairs, correct_line_idx

def create_mask(src,tgt):
    src_seq_len=src.shape[0]
    tgt_seq_len=tgt.shape[0]
    
    tgt_mask=generate_square_mask(tgt_seq_len)
    src_mask=torch.zeros((src_seq_len,src_seq_len),device=device).type(torch.bool)
    
    src_padding_mask=(src==pad_idx).transpose(0,1)
    tgt_padding_mask=(tgt==pad_idx).transpose(0,1)
    
    return src_mask,tgt_mask,src_padding_mask,tgt_padding_mask   


def indexesFromSentence(lang,sentence):
    return [lang.char2index[char] for char in sentence]

def tensorFromSentence(lang,sentence):
    indexes=indexesFromSentence(lang,sentence)
    indexes.append(eos_idx)
    indexes.insert(0,bos_idx)
    return torch.tensor(indexes,dtype=torch.long,device=device).view(-1,1)

def collate_fn(pair):
    src_batch, tgt_batch = [], [] 
    src_list=[input_lang.char2index[char.lower()] for char in pair[0]]
    src_list.insert(0,bos_idx)
    src_list.append(eos_idx)
        
    tgt_list=[output_lang.char2index[char.lower()] for char in pair[1]]
    tgt_list.insert(0,bos_idx)
    tgt_list.append(eos_idx)
        
    src_batch.append(torch.tensor(src_list,dtype=torch.long,device=device).view(-1,1))
    tgt_batch.append(torch.tensor(tgt_list,dtype=torch.long,device=device).view(-1,1))
        
    src_batch=pad_sequence(src_batch,padding_value=pad_idx)
    tgt_batch=pad_sequence(tgt_batch,padding_value=pad_idx)
    
    return src_batch, tgt_batch


input_lang, output_lang, pairs, correct_line_idx=prepareData(src_lang,tgt_lang,reverse=False)

nr_pairs_subset=10**(4) 
#nr_pairs_subset=len(pairs) 
pairs_subset_idx=np.random.choice(range(len(pairs)),size=nr_pairs_subset,replace=True)
pairs_subset=[pairs[idx] for idx in pairs_subset_idx] 

percentage_train=0.9
training_idx=np.random.choice(range(nr_pairs_subset),size=int(nr_pairs_subset*percentage_train),replace=False)
testing_idx=[idx for idx in range(nr_pairs_subset) if idx not in training_idx]

pairs_train=[pairs_subset[idx] for idx in training_idx]
pairs_test=[pairs_subset[idx] for idx in testing_idx]
nr_test=len(pairs_test)
nr_train=len(pairs_train)

src_vocab_size=input_lang.n_chars
tgt_vocab_size=output_lang.n_chars
emb_size=512
nhead=8
#ffn_hid_dim=256
batch_size=32
num_encoder_layers=4
num_decoder_layers=4


transformer=seq2seqTransformer(num_encoder_layers,num_decoder_layers,emb_size,nhead,src_vocab_size,tgt_vocab_size,
                                dim_feedforward=512,dropout=0.1)

for p in transformer.parameters():
    if p.dim()>1:
        nn.init.xavier_uniform_(p)
        
transformer=transformer.to(device)

criterion=nn.CrossEntropyLoss(ignore_index=pad_idx)
#lr=10**(-4)
lr=0
optimizer=torch.optim.Adam(transformer.parameters(),lr=lr,betas=(0.9,0.98),eps=10**(-9),amsgrad=True)
epochs=2
epoch=1
printing_iter=10**(2) 

all_folds_idx=[]
train_losses=[]
k_folds=5
overall_accuracy=[]
count_train=0


for epoch in range(1,epochs+1):
    rem_data_indexes=training_idx
    flag=1
    start=time.time()
    while flag==1:
      nr_data=len(rem_data_indexes)
      if nr_data<=k_folds and nr_data>0:
        folds_idx=rem_data_indexes
        #rem_data_indexes=folds_idx
        flag=0
      if nr_data>k_folds:
        folds_idx=np.random.choice(rem_data_indexes,size=k_folds,replace=False).tolist()
        rem_data_indexes=[idx for idx in rem_data_indexes if idx not in folds_idx]
      
      fold_loss=0
      for i in folds_idx:
        src, tgt=collate_fn(pairs[i])
        optimizer=torch.optim.Adam(transformer.parameters(),lr=lr,betas=(0.9,0.98),eps=10**(-9))
        count_train+=1
        step_num=count_train
        lr=1/math.sqrt(emb_size)*min(1/math.sqrt(step_num+1),(step_num+1)*warmup_steps**(-1.5))
        optimizer.zero_grad() 
        train_loss=0
        src=src[:,0,:].to(device)
        tgt=tgt[:,0,:].to(device)
        
        tgt_input=tgt[:-1,:]
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask=create_mask(src,tgt_input)
        
        output=transformer(src,tgt_input,src_mask,tgt_mask,src_padding_mask,tgt_padding_mask,src_padding_mask)
        
        tgt_out=tgt[1:,:]
        tgt_out=tgt_out.reshape(-1)
        output=output.reshape(-1,output.shape[-1])
        fold_loss+=criterion(output,tgt_out) 

      fold_loss.backward()
      optimizer.step()
        
      train_losses.append(fold_loss.item()/len(folds_idx))

      if (count_train%printing_iter==0 and count_train>0):
        print("[%d] Started fold:"%(epoch))
        file1.write("[%d] Started fold:\n"%(epoch))
        print(folds_idx)
        file1.write("Folds are: " + ",".join([str(idx) for idx in folds_idx])+"\n") 
        print("Fold loss: %.3f \t Avg fold loss: %.3f \t time for fold is %s"%(fold_loss/len(folds_idx),np.mean(train_losses),timming(start)))
        file1.write("Fold loss: %.3f \t Avg fold loss: %.3f \t time for fold is %s \n"%(fold_loss/len(folds_idx),np.mean(train_losses),timming(start)))
        print("Training completion: %.2f%% \t nr of remiaing indexes in training: %d"%((100*epoch*(nr_train-len(rem_data_indexes)))/(nr_train*epochs),len(rem_data_indexes)))
        file1.write("Training completion: %.2f%% \t nr of remiaing indexes in training: %d"%((100*epoch*(nr_train-len(rem_data_indexes)))/(nr_train*epochs),len(rem_data_indexes)))
        print("--------------------------------------------")
        file1.write("--------------------------------------------\n")

    accuracy=[]
    input_names=[]
    output_names=[]
    target_names=[]
    correct_predicted_names=[]
    correct_actual_names=[]
    correct_latin_names=[]
    incorrect_predicted_names=[]
    incorrect_actual_names=[]
    incorrect_latin_names=[]
    count_test=0
    u=0
    with torch.no_grad():
      rem_data_indexes=testing_idx
      flag=1
      while flag==1:
        nr_data=len(rem_data_indexes)
        if nr_data<=k_folds and nr_data>0:
          folds_idx=rem_data_indexes
          #rem_data_indexes=folds_idx
          flag=0
        if nr_data>k_folds:
          folds_idx=np.random.choice(rem_data_indexes,size=k_folds,replace=False).tolist()
          rem_data_indexes=[idx for idx in rem_data_indexes if idx not in folds_idx]

        for i in folds_idx:
          src_name=pairs[i][0]
          tgt_name=pairs[i][1]
          src_list=[input_lang.char2index[char.lower()] for char in src_name]
          src_list.insert(0,bos_idx)
          src_list.append(eos_idx)
          src=torch.tensor(src_list,dtype=torch.long,device=device).view(-1,1)
          src=src.view(src.shape[0],1,1)
          #src=pad_sequence(src,padding_value=pad_idx)

          src=src[:,0].to(device)
          src_seq_len=src.shape[0]
          src_mask=torch.zeros((src_seq_len,src_seq_len),device=device).type(torch.bool)
          memory=transformer.encode(src,src_mask)
          ys=torch.ones(1,1).fill_(bos_idx).type(torch.long).to(device)
          num_tokens=src.shape[0]
          max_len=num_tokens+5
          memory=memory.to(device)

          for j in range(max_len-1):
            tgt_mask=(generate_square_mask(ys.size(0))).type(torch.bool).to(device)
            out=transformer.decode(ys,memory,tgt_mask)
            out=out.transpose(0,1) 
            out=transformer.generator(out[:,-1])
            prob=F.softmax(out,dim=1)
            _, next_char=torch.max(prob,dim=1)
            next_char=next_char.item()

            ys=torch.cat([ys,torch.ones(1,1).type_as(src.data).fill_(next_char)],dim=0) 
            if next_char==eos_idx:
              break

          ys=ys.flatten().cpu().numpy()
          output_name="".join([output_lang.index2char[idx] for idx in ys]).replace("<bos>","").replace("<eos>","")
          tgt_name="".join([char for char in tgt_name]).replace("<bos>","").replace("<eos>","")
          #output_name=output_name.split(" ")

          output_name=output_name.split(" ")
          tgt_name=tgt_name.split(" ")
          output_names.append(output_name)
          target_names.append(tgt_name)
          input_names.append(src_name)
          
        count_test+=1

        if output_name==tgt_name:
          u+=1
          temp=" ".join([name for name in input_names[-1]])
          correct_latin_names.append(temp)
          temp=" ".join([name for name in output_names[-1]])
          correct_predicted_names.append(temp)
          temp=" ".join([name for name in target_names[-1]])
          correct_actual_names.append(temp)
        else: 
          temp=" ".join([name for name in input_names[-1]])
          incorrect_latin_names.append(temp)
          temp=" ".join([name for name in output_names[-1]])
          incorrect_predicted_names.append(temp)
          temp=" ".join([name for name in target_names[-1]])
          incorrect_actual_names.append(temp)

        overall_accuracy.append(100*u/(i+1))

        if (count_test%printing_iter==0 and count_test>0):
          
          print("[%.2f%%] Testing completion:%.2f%% \t Accuracy avg so far:%.5f%% \t time:%s"%
                (100*epoch/epochs, 100*epoch*(nr_test-len(rem_data_indexes))/(nr_test*epochs), overall_accuracy[-1], timming(start)))
          file1.write("[%.2f%%] Testing completion:%.2f%% \t Accuracy avg so far:%.5f%% \t time:%s \n"%
                (100*epoch/epochs, 100*epoch*(nr_test-len(rem_data_indexes))/(nr_test*epochs), overall_accuracy[-1], timming(start)))
          print("--------------------------------------------")
          file1.write("-------------------------------------------- \n") 


    print("[%.2f%%] Training loss=%.7f \t Testing mean accuracy:%.2f%% \t time=%s"%(100*epoch/epochs,train_losses[-1],overall_accuracy[-1],timming(start)))
    file1.write("[%.2f%%] Training loss=%.7f \t Testing mean accuracy:%.2f%% \t time=%s \n"%(100*epoch/epochs,train_losses[-1],overall_accuracy[-1],timming(start)))

    torch.save(transformer,"/content/drive/MyDrive/WunderSchildt/Translation/All_saved_models/Char CV rus transformer, epoch %d, names %d, folds %d,embedding %d, loss %.4f, layers %d, acc %.2f%%.pth"%
           (epoch,nr_pairs_subset,k_folds,emb_size,np.mean(train_losses),num_encoder_layers,100*u/(count_test*(len(folds_idx)))))
    
    correct_names=pd.DataFrame()
    correct_names["correct predicted names"]=correct_predicted_names
    correct_names["actual names"]=correct_actual_names
    correct_names["latin names"]=correct_latin_names
    correct_names.to_csv("/content/drive/MyDrive/WunderSchildt/Translation/Predictions/Translated Correct Names by char CV rus transformer, epoch %d, names %d, folds %d,embedding %d, loss %.4f, layers %d, acc %.2f%%.csv"%
                  (epoch,nr_pairs_subset,k_folds,emb_size,np.mean(train_losses),num_encoder_layers,overall_accuracy[-1]))
    
    incorrect_names=pd.DataFrame()
    incorrect_names["incorrect predicted names"]=incorrect_predicted_names
    incorrect_names["actual names"]=incorrect_actual_names
    incorrect_names["latin names"]=incorrect_latin_names
    incorrect_names.to_csv("/content/drive/MyDrive/WunderSchildt/Translation/Predictions/Translated Incorrect Names by char CV rus transformer, epoch %d, names %d,folds %d,embedding %d, loss %.4f, layers %d, acc %.2f%%.csv"%
                  (epoch,nr_pairs_subset,k_folds,emb_size,np.mean(train_losses),num_encoder_layers,overall_accuracy[-1]))

      
file1.close()
plt.title("Training loss")
plt.plot(train_losses,label="loss")
plt.xlabel("Number of names")
plt.ylabel("Loss")
plt.legend() 
plt.show()
plt.title("Accuracy")
plt.plot(overall_accuracy,label="accuracy")
plt.xlabel("Number of names")
plt.ylabel("Accuracy")
plt.legend()
plt.show()
train_losses_avg=[sum(train_losses[:(i+1)])/(i+1) for i in range(len(train_losses)-1)]
plt.title("Training loss avg")
plt.plot(train_losses_avg,label="avg loss")
plt.xlabel("Number of names")
plt.ylabel("Avg Loss")
plt.legend()
plt.show()




reading lines...
Read 401044 name pairs
Counting characters:
counted characters:
En 164
Rus 39
[1] Started fold:
[9262, 7969, 6469, 3448, 9546]
Fold loss: 3.429 	 Avg fold loss: 3.737 	 time for fold is 0m 7s
Training completion: 0.31% 	 remiaing indexes in training: 8945
--------------------------------------------
[1] Started fold:
[1900, 4443, 76, 3248, 864]
Fold loss: 3.563 	 Avg fold loss: 3.531 	 time for fold is 0m 14s
Training completion: 0.58% 	 remiaing indexes in training: 8895
--------------------------------------------
[1] Started fold:
[2813, 5757, 3357, 4494, 4728]
Fold loss: 3.294 	 Avg fold loss: 3.450 	 time for fold is 0m 20s
Training completion: 0.86% 	 remiaing indexes in training: 8845
--------------------------------------------
[1] Started fold:
[3323, 5030, 8016, 4492, 9979]
Fold loss: 3.209 	 Avg fold loss: 3.397 	 time for fold is 0m 26s
Training completion: 1.14% 	 remiaing indexes in training: 8795
--------------------------------------------
[1] Started f

KeyboardInterrupt: ignored